In [ ]:
!pip install pandas
!pip install numpy
!pip install seaborn
!pip install matplotlib
!pip install scikit-learn
!pip install scipy
!pip install gdown


In [2]:

# Import portion of a package
import matplotlib.pyplot as plt  # Most common visualization package that a lot of others are based on

# Import full packages under custom name
import numpy as np  # Common package for numerical methods
import pandas as pd  # Common package for data storeage/manipulation
import seaborn as sns  # Common package for statistical visualizations

# Import portion of a package
import scipy.stats as stats
from sklearn.impute import SimpleImputer as Imputer  # Specific function from common machine learning package\

#more packages
import gdown


In [3]:


# File ID from your link (assuming this is the cleaned dataset)
file_id = "13elVDJJ6JUH2gsj36CxPjkLryE2rnl-r"
url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
gdown.download(url, "flight_data.csv", quiet=False)

# Now load it
flight_data = pd.read_csv("flight_data.csv")
print(flight_data.shape)
print(flight_data.head())

Downloading...
From (original): https://drive.google.com/uc?id=13elVDJJ6JUH2gsj36CxPjkLryE2rnl-r
From (redirected): https://drive.google.com/uc?id=13elVDJJ6JUH2gsj36CxPjkLryE2rnl-r&confirm=t&uuid=f2f0f974-859d-4241-a8ce-7be7b2d4f2fa
To: /content/flight_data.csv
100%|██████████| 1.32G/1.32G [00:11<00:00, 115MB/s]


(7546988, 32)
   YEAR  QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK  MKT_CARRIER_AIRLINE_ID  \
0  2024        1      1             1            1                   19393   
1  2024        1      1             1            1                   19393   
2  2024        1      1             1            1                   19393   
3  2024        1      1             1            1                   19393   
4  2024        1      1             1            1                   19393   

   ORIGIN_AIRPORT_ID  ORIGIN_AIRPORT_SEQ_ID ORIGIN_CITY_NAME  DEST_AIRPORT_ID  \
0              10140                1014005  Albuquerque, NM            10423   
1              10140                1014005  Albuquerque, NM            10423   
2              10140                1014005  Albuquerque, NM            10800   
3              10140                1014005  Albuquerque, NM            10821   
4              10140                1014005  Albuquerque, NM            11259   

   ...  CANCELLATION_CODE CRS_

In [4]:

df = flight_data.copy()
print (df.head())
print(df.columns.tolist())

   YEAR  QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK  MKT_CARRIER_AIRLINE_ID  \
0  2024        1      1             1            1                   19393   
1  2024        1      1             1            1                   19393   
2  2024        1      1             1            1                   19393   
3  2024        1      1             1            1                   19393   
4  2024        1      1             1            1                   19393   

   ORIGIN_AIRPORT_ID  ORIGIN_AIRPORT_SEQ_ID ORIGIN_CITY_NAME  DEST_AIRPORT_ID  \
0              10140                1014005  Albuquerque, NM            10423   
1              10140                1014005  Albuquerque, NM            10423   
2              10140                1014005  Albuquerque, NM            10800   
3              10140                1014005  Albuquerque, NM            10821   
4              10140                1014005  Albuquerque, NM            11259   

   ...  CANCELLATION_CODE CRS_ELAPSED_TIME  

In [ ]:
"""
minimize sum of (epsilon?) (frequency * average * passengers)
st.
    ...

optimization problem in words
- frequency = probability of flight being delayed
- average = expected delay time (in minutes)
- passengers = number of passengers taking that flight
- possible constraints for passengers:
  - arrival/depaarture time being between certain times
  - flight duration being shorter than x hours
  - groups of passengers staying together (ie families)
  - max number of extra seats on flights
  - need to get to destination by a certain time
  - airline?
  - any connecting flights must have at least a 1 hour buffer for connecting purposes






info needed about the set of flights:
- number of seats available (this is a made up number)
- frequency
- average
- arrival/departure time/day/month/year
- flight duration
- destination/origin airport/city
- airline

"""

In [32]:
#calculating average and frequency of delay

delay_cols = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY']

# Function to compute probability of delay and average delay for each flight
def compute_delay_stats(group):
    # Boolean series: True if any delay > 0 in that row
    delayed = group[delay_cols].gt(0).any(axis=1)

    # Probability of delay
    prob_delay = delayed.sum() / len(group)

    # Average delay (only for delayed rows)
    if delayed.sum() > 0:
        avg_delay = group.loc[delayed, delay_cols].sum(axis=1).mean()
    else:
        avg_delay = 0.0

    return pd.Series({'freq_delay': prob_delay, 'avg_delay': avg_delay})

# Group by origin and destination airport to calculate delay stats
delay_stats = df.groupby(['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']).apply(compute_delay_stats).reset_index()
print (delay_stats.head())



   ORIGIN_AIRPORT_ID  DEST_AIRPORT_ID  freq_delay  avg_delay
0              10135            10397    0.121927  65.766129
1              10135            10693    0.085714  44.333333
2              10135            11057    0.117496  78.481752
3              10135            11292    0.197674  42.176471
4              10135            11697    0.148148  44.666667


/tmp/ipython-input-2552609627.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  delay_stats = df.groupby(['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']).apply(compute_delay_stats).reset_index()


In [34]:
#making set of top 20 flights

import pandas as pd
import numpy as np

# Assuming the dataframe is already loaded
# Group by 'ORIGIN_CITY_NAME' and 'DEST_AIRPORT_ID' and count the occurrences
top_routes = df.groupby(['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']).agg(
    origin_city=('ORIGIN_CITY_NAME', 'first'),
    destination_city=('DEST_CITY_NAME', 'first'),
    count=('ORIGIN_CITY_NAME', 'size'),
    avg_duration=('ACTUAL_ELAPSED_TIME', 'mean'),
    most_common_departure=('CRS_DEP_TIME', lambda x: x.mode()[0] if not x.mode().empty else np.nan), #arrival time will be calculated by adding elapsed time to arrival
    most_common_airline=('MKT_CARRIER_AIRLINE_ID', lambda x: x.mode()[0] if not x.mode().empty else np.nan)
).reset_index()

# Add a "made-up" number of seats available (for now I will assume random values, e.g. between 100 and 300)
top_routes['seats_available'] = np.random.randint(1, 6, size=len(top_routes))

# Sort by count in descending order and take the top 20
top_20_routes = top_routes.sort_values(by='count', ascending=False).head(40) # can adjust number for sets of flights, did 40 bc the top 20 is the top 10 city1-city2 and vice versa

# Merge with your top_20_routes dataframe
top_20_routes = top_20_routes.merge(delay_stats, on=['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID'], how='left')

top_20_routes['origin_city'] = top_20_routes['origin_city'].str.split(',').str[0] #extracting city from city,state
top_20_routes['destination_city'] = top_20_routes['destination_city'].str.split(',').str[0]


#calculating the most common city to use as starting point for the group of passengers
origin_counts = top_20_routes['ORIGIN_AIRPORT_ID'].astype(int).value_counts()
most_common_origin_id = origin_counts.idxmax()

print("Most common origin airport ID:", most_common_origin_id) # it is 12892 (freq of 5) which is LA

# Display the top 20 routes with all the requested details
print(top_20_routes[['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'origin_city', 'destination_city', 'avg_duration', 'most_common_departure',
                       'most_common_airline','seats_available', 'freq_delay', 'avg_delay']])


Most common origin airport ID: 12892
    ORIGIN_AIRPORT_ID  DEST_AIRPORT_ID      origin_city destination_city  \
0               13830            12173          Kahului         Honolulu   
1               12173            13830         Honolulu          Kahului   
2               12892            14771      Los Angeles    San Francisco   
3               14771            12892    San Francisco      Los Angeles   
4               12953            13930         New York          Chicago   
5               13930            12953          Chicago         New York   
6               12892            12889      Los Angeles        Las Vegas   
7               12889            12892        Las Vegas      Los Angeles   
8               12478            12892         New York      Los Angeles   
9               12892            12478      Los Angeles         New York   
10              11278            10721       Washington           Boston   
11              10721            11278           Bo

In [37]:
#dictionary:
#'ORIGIN-DEST-x': [origin airport, dest airport, flight time, departure time, seats available, airline, delay freq,
#delay avg]
#-x because the departure time can be diff if the origin-dest flight has multiple


# print (d)
import random
simulated_dict = {}

for _, row in top_20_routes.iterrows():

    # Key base like "New York-Los Angeles"
    base_key = f"{row['origin_city']}-{row['destination_city']}"

    # Random number of simulated flights (e.g. between 1 and 5)
    num_simulated_flights = random.randint(1, 5)

    for i in range(1, num_simulated_flights + 1):

        # Random departure time in HHMM (0000–2359 but realistic ranges optional)
        allowed_times = []

        for hour in range(24):          # 0 → 23
            for minute in [0, 30]:      # :00 and :30
                time_value = hour * 100 + minute
                allowed_times.append(time_value)


        random_time = random.choice(allowed_times)
        # Random seats available
        random_seats = random.randint(1,5)

        # Build key with -1, -2, -3 suffix
        key = f"{base_key}-{i}"

        # Store all data exactly like your structure
        simulated_dict[key] = [
            row['origin_city'],
            row['destination_city'],
            row['ORIGIN_AIRPORT_ID'],
            row['DEST_AIRPORT_ID'],
            row['avg_duration'],
            row['freq_delay'],
            row['avg_delay'],
            random_time,
            row['most_common_airline'],
            random_seats
        ]

dictionary_df = pd.DataFrame(simulated_dict, index = ['origin_city', 'destination_city','origin_airport', 'destination_airport', 'expected_duration', 'freq_delay', 'avg_delay','departure(HHMM)', 'airline', 'num_seats'])
dictionary_df



,Kahului-Honolulu-1,Kahului-Honolulu-2,Kahului-Honolulu-3,Honolulu-Kahului-1,Honolulu-Kahului-2,Honolulu-Kahului-3,Honolulu-Kahului-4,Honolulu-Kahului-5,Los Angeles-San Francisco-1,Los Angeles-San Francisco-2,...,Seattle-Portland-2,Seattle-Portland-3,Seattle-Portland-4,Portland-Seattle-1,Seattle-Phoenix-1,Seattle-Phoenix-2,Seattle-Phoenix-3,Phoenix-Seattle-1,Phoenix-Seattle-2,Phoenix-Seattle-3
origin_city,Kahului,Kahului,Kahului,Honolulu,Honolulu,Honolulu,Honolulu,Honolulu,Los Angeles,Los Angeles,...,Seattle,Seattle,Seattle,Portland,Seattle,Seattle,Seattle,Phoenix,Phoenix,Phoenix
destination_city,Honolulu,Honolulu,Honolulu,Kahului,Kahului,Kahului,Kahului,Kahului,San Francisco,San Francisco,...,Portland,Portland,Portland,Seattle,Phoenix,Phoenix,Phoenix,Seattle,Seattle,Seattle
origin_airport,13830,13830,13830,12173,12173,12173,12173,12173,12892,12892,...,14747,14747,14747,14057,14747,14747,14747,14107,14107,14107
destination_airport,12173,12173,12173,13830,13830,13830,13830,13830,14771,14771,...,14057,14057,14057,14747,14107,14107,14107,14747,14747,14747
expected_duration,42.645562,42.645562,42.645562,42.447713,42.447713,42.447713,42.447713,42.447713,87.731371,87.731371,...,54.896401,54.896401,54.896401,60.248529,170.186648,170.186648,170.186648,185.474666,185.474666,185.474666
freq_delay,0.090277,0.090277,0.090277,0.088104,0.088104,0.088104,0.088104,0.088104,0.298243,0.298243,...,0.165854,0.165854,0.165854,0.115041,0.175538,0.175538,0.175538,0.14777,0.14777,0.14777
avg_delay,19.352046,19.352046,19.352046,23.290732,23.290732,23.290732,23.290732,23.290732,46.407837,46.407837,...,39.50449,39.50449,39.50449,54.840989,40.362922,40.362922,40.362922,38.969159,38.969159,38.969159
departure(HHMM),0,600,530,100,830,1130,700,2230,130,1300,...,2000,100,800,1400,1730,1530,700,1200,830,1130
airline,19690,19690,19690,19690,19690,19690,19690,19690,19977,19977,...,19930,19930,19930,19930,19930,19930,19930,19930,19930,19930
num_seats,2,3,5,3,5,3,1,4,3,5,...,3,1,5,5,2,1,5,1,5,4


In [38]:
#making passenger df with passenger preferences
def generate_passengers(n_passengers, dictionary_df):

    passengers = []

    # extract valid airports from your simulated dictionary
    valid_destinations = dictionary_df.loc['destination_city'].tolist()

    # random grouping (families)
    # groups of size 1–4
    group_sizes = []
    remaining = n_passengers

    while remaining > 0:
        size = random.randint(1, min(4, remaining))
        group_sizes.append(size)
        remaining -= size

    group_id = 1
    passenger_id = 1

    for size in group_sizes:
        for _ in range(size):

            passenger = {}
            passenger['passenger_id'] = passenger_id
            passenger['group_id'] = group_id

            # random origin/destination
            passenger['origin_airport'] = 'Los Angeles'
            passenger['destination_airport'] = random.choice([d for d in valid_destinations])

            # random constraints
            passenger['latest_arrival'] = random.choice([h*100 + m for h in range(48) for m in [0,30]]) # time elapsed from origin time
            # passenger['max_duration'] = random.choice([60, 90, 120, 180, 240, 300])  #commented out bc might not need constraint of max duration of a flight

            # airline preference (20% chance of having one)
            airline_pref = random.choice(dictionary_df.loc['airline'].tolist())
            passenger['airline_pref'] = airline_pref if random.random() < 0.2 else None

            # connecting buffer requirement (always 60 minutes)
            # passenger['min_connection_time'] = 60

            passengers.append(passenger)
            passenger_id += 1

        group_id += 1

    return pd.DataFrame(passengers)

passengers_df = generate_passengers(10, dictionary_df)
print(passengers_df)

   passenger_id  group_id origin_airport destination_airport  latest_arrival  \
0             1         1    Los Angeles             Chicago            3100   
1             2         2    Los Angeles             Seattle            1630   
2             3         3    Los Angeles              Boston             800   
3             4         3    Los Angeles            New York            1200   
4             5         4    Los Angeles            Honolulu             600   
5             6         4    Los Angeles             Atlanta            1430   
6             7         5    Los Angeles         Los Angeles            3130   
7             8         5    Los Angeles             Kahului             430   
8             9         6    Los Angeles              Denver            2430   
9            10         7    Los Angeles             Kahului            3100   

   airline_pref  
0           NaN  
1       19790.0  
2       19790.0  
3           NaN  
4           NaN  
5          

In [39]:
#formulating optimization problem
import cvxpy as cp

passengers = passengers_df['passenger_id'].tolist()
flights = dictionary_df.columns.tolist()

num_passengers = len(passengers)
num_flights = len(flights)

# decision variable: x[p, f] = 1 if passenger p is assigned to flight f
x = cp.Variable((num_passengers, num_flights), boolean=True)

# Objective: use freq_delay and avg_delay from dictionary_df (EDWARD TBD)
freq_delay = np.array([dictionary_df[f]['freq_delay'] for f in flights])
avg_delay = np.array([dictionary_df[f]['avg_delay'] for f in flights])

# Element-wise multiplication, broadcast over passengers
objective = cp.Minimize(cp.sum(cp.multiply(x, freq_delay * avg_delay)))

# Constraints list
constraints = []

# Airline preference constraint
for p_idx, p in enumerate(passengers):
    airline_pref = passengers_df.loc[passengers_df['passenger_id']==p, 'airline_pref'].values[0]
    if airline_pref is not None:
        for f_idx, f in enumerate(flights):
            if dictionary_df[f]['airline'] != airline_pref:
                constraints.append(x[p_idx, f_idx] == 0)

#arrive by certain time constraint
for p_idx, p in enumerate(passengers):
    latest = passengers_df.loc[passengers_df['passenger_id']==p, 'latest_arrival'].values[0]
    for f_idx, f in enumerate(flights):
        dep = dictionary_df[f]['departure(HHMM)']
        dur = dictionary_df[f]['expected_duration']
        arrival_time = dep + int(dur)  # approximate HHMM
        if arrival_time > latest:
            constraints.append(x[p_idx, f_idx] == 0)

#seat availability constraint
for f_idx, f in enumerate(flights):
    seats = dictionary_df[f]['num_seats']
    constraints.append(cp.sum(x[:, f_idx]) <= seats)

# Group constraint (on same flight)
groups = passengers_df['group_id'].unique()
for g in groups:
    group_passengers = passengers_df.loc[passengers_df['group_id']==g].index.tolist()
    if len(group_passengers) > 1:
        # auxiliary variable for this group on each flight
        y = cp.Variable(num_flights, boolean=True)
        for f_idx in range(num_flights):
            # enforce x[group_member, f] == y[f]
            for p_idx in group_passengers:
                constraints.append(x[p_idx, f_idx] == y[f_idx])
        # ensure the group is assigned to exactly one flight
        constraints.append(cp.sum(y) == 1)

# Solve MILP
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.GLPK_MI)

# Extract assignments
assignment_matrix = x.value
assigned_pairs = []
for p_idx, p in enumerate(passengers):
    for f_idx, f in enumerate(flights):
        if assignment_matrix[p_idx, f_idx] > 0.5:
            assigned_pairs.append((p, f))

assignments_df = pd.DataFrame(assigned_pairs, columns=['passenger_id','flight'])
print(assignments_df)

/usr/local/lib/python3.12/dist-packages/cvxpy/reductions/solvers/solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


TypeError: 'NoneType' object is not subscriptable